[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/matlogica/QuantBench/main?filepath=2AssetAutoCallable/results/results_viewer.ipynb)

# Phoenix Autocallable Note - Accuracy Analysis

This notebook analyzes the effects of Monte Carlo path count and smoothing parameter on the accuracy of pricing and risk sensitivities for the Phoenix Autocallable Note. 

We'll compare:
1. **Base implementation** (standard bump-and-revalue)
2. **Smoothed implementation** (using smoothed digital functions)
3. **AADC implementation** (Algorithmic Adjoint Differentiation for C++)

We'll examine how these approaches converge as we increase the number of Monte Carlo paths, and analyze the computational efficiency trade-offs.

In [3]:
!pip install pandas numpy seaborn ipywidgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import glob
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12


## Loading and Processing the Data Files

We'll load all the CSV files and organize them by smoothing parameter and path count.

In [5]:
from viewer_tools import *

# Load all data files
data_by_params = load_all_data()

# Extract unique smoothing parameters and path counts
smoothing_params = sorted(set(param[0] for param in data_by_params.keys()))
path_counts = sorted(set(param[1] for param in data_by_params.keys()))
ladders = sorted(set(param[2] for param in data_by_params.keys()))

print(f"Loaded {len(data_by_params)} data files")
print(f"Smoothing parameters: {smoothing_params}")
print(f"Path counts: {path_counts}")
print(f"Ladders : {ladders}")

Loaded 360 data files
Smoothing parameters: [0.0, 0.1, 0.4, 0.8, 1.6, 2.4, 3.5]
Path counts: [10000, 50000, 100000, 500000, 1000000, 10000000]
Ladders : ['corr', 'corr-noisy', 'spot1', 'spot1-noisy', 'spot2', 'spot2-noisy', 'vol1', 'vol1-noisy', 'vol2', 'vol2-noisy']


## Establishing the Reference Case (10M Paths)

We'll use the 10M path simulation as our reference "ground truth" for analyzing convergence.

In [6]:
reference_key = {}
reference_data = {}
for ladder in ladders:
    # Find the reference case (10M paths with 0.1 smoothing)
    reference_key[ladder] = next((k for k in data_by_params.keys() if k[1] == 10000000 and k[2] == ladder), None)
    
    if reference_key[ladder]:
        reference_data[ladder] = data_by_params[reference_key[ladder]]
        print(f"Reference case for {ladder}: Smoothing={reference_key[ladder][0]}, Paths={reference_key[ladder][1]}")
    else:
        print(f"Reference case with 10M paths for ladder {ladder} not found. Using the highest path count as reference.")
        max_paths = max(k[1] for k in data_by_params.keys() if k[2] == ladder)
        reference_key[ladder] = next((k for k in data_by_params.keys() if k[1] == max_paths and k[2] == ladder), None)
        reference_data[ladder] = data_by_params[reference_key[ladder]]
        print(f"Using reference: Smoothing={reference_key[ladder][0]}, Paths={reference_key[ladder][1]}")

Reference case for corr: Smoothing=0.8, Paths=10000000
Reference case for corr-noisy: Smoothing=0.8, Paths=10000000
Reference case for spot1: Smoothing=0.8, Paths=10000000
Reference case for spot1-noisy: Smoothing=0.8, Paths=10000000
Reference case for spot2: Smoothing=0.8, Paths=10000000
Reference case for spot2-noisy: Smoothing=0.8, Paths=10000000
Reference case for vol1: Smoothing=0.8, Paths=10000000
Reference case for vol1-noisy: Smoothing=0.8, Paths=10000000
Reference case for vol2: Smoothing=0.8, Paths=10000000
Reference case for vol2-noisy: Smoothing=0.8, Paths=10000000


## Interactive Data Explorer

Let's create interactive widgets to explore the data across different dimensions.

In [7]:

# Create the interactive explorer
create_interactive_explorer(ladders, smoothing_params, path_counts, reference_data, reference_key, data_by_params)

Output()

## Convergence Analysis by Smoothing Parameter

Let's compare how different smoothing parameters affect convergence.

## Computational Efficiency Analysis

Let's analyze the computational efficiency of each method by comparing the accuracy achieved per unit of computation time.

## Summary of Key Findings

Based on the analysis, we can draw the following conclusions:

1. **Convergence Rate**: 
   - The smoothed implementation converges faster to the reference solution than the base implementation
   - The AADC implementation provides the most consistent results across different path counts

2. **Computational Efficiency**:
   - AADC provides the best accuracy per unit of computation time
   - The smoothed implementation offers a good balance between implementation complexity and convergence speed

3. **Effect of Smoothing Parameter**:
   - A smaller smoothing parameter (e.g., 0.1) generally provides better accuracy than larger values
   - The optimal smoothing parameter may depend on the specific sensitivity being calculated

4. **Path Count Requirements**:
   - For accurate correlation sensitivity, at least 100,000 paths are recommended for the base implementation
   - The smoothed implementation can achieve similar accuracy with fewer paths
   - AADC provides good accuracy even with relatively few paths

Overall, these results demonstrate the benefits of both smoothing techniques and adjoint algorithmic differentiation for calculating sensitivities of complex structured products like the Phoenix Autocallable Note.